# Training an Object Detection model using Detectron2

This notebook demonstrates how to train a [Detectron2](https://github.com/facebookresearch/detectron2/) model on object detection datasets and produce predictions required to run cleanlab's tutorial on detecting label errors in object detection data.  Note that this notebook fits the model to an entire training set and produces predictions on a held-out validation set. Thus these predictions are only *out-of-sample* for the validation data, and should ideally *only* be used to find mislabeled images amongst the validation set. To instead find mislabeled images amongst an entire dataset, see the analogous notebook in this folder which uses K-fold cross-validation to produce out-of-sample predictions for every image in the dataset.

In object detection data, each image is annotated with multiple bounding boxes.  Each bounding box surrounds a physical object within an image scene, and is annotated with a given class label. Using this labeled data, we train a model to predict the locations and classes of objects in an image. The trained model can subsequently be used to identify mislabeled images, which when corrected, allow you to train an even better model without changing your training code! 

Here we will fit a state-of-the-art neural network trained starting from a pretrained [X-101](https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md#imagenet-pretrained-models) network backbone. First let's import the required packages and download the data.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cleanlab/examples/blob/master/object_detection/detectron2_training.ipynb)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import pickle
# import some common libraries
import numpy as np
import os, json, cv2, random
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances


In [ ]:
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/DATASET_annotations/instances_val2017_5labels.json"
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/DATASET_annotations/instances_train2017_5labels.json"
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/labels.pkl"

Before you begin training on a custom dataset, be sure to review the COCO dataset guidelines for formatting your data, which can be found on their [website](https://cocodataset.org/#format-data).

Here we use a custom dataset named "my_dataset" for training. A subset of the labels ["car", "chair", "cup", "person", and "traffic light"] are used for training and detecting errors in this notebook.

In [ ]:
IMAGE_PATH = "../data/coco/"
TRAIN_PATH = os.path.join(IMAGE_PATH,"train2017")
VAL_PATH = os.path.join(IMAGE_PATH,"val2017")
register_coco_instances("my_dataset_train", {}, "instances_train2017_5labels.json",
                        TRAIN_PATH)
register_coco_instances("my_dataset_val", {}, "instances_val2017_5labels.json",
                        VAL_PATH)


We define the configuration settings for training an object detection model using Detectron2. The model architecture used in this example is "faster_rcnn_X_101_32x8d_FPN_3x" from the COCO-Detection model zoo. The training data is specified by the "my_dataset_train" dataset and validation data is specified by the "my_dataset_val" dataset which refer to COCO2017 train and val containing only the subset of labels specified before.

Here the number of worker threads is set to 2 and the batch size is set to 2. The learning rate and maximum number of iterations are also specified. You'll want to tinker with these values to get the best performance for your own data.
The model is initialized from the COCO-Detection model zoo and the output directory for the trained model is created. Finally, this configuration is passed to the DefaultTrainer class for training the object detection model.

<strong>Note:</strong> The number of iterations was set based on [early stopping.](https://en.wikipedia.org/wiki/Early_stopping#:~:text=In%20machine%20learning%2C%20early%20stopping,training%20data%20with%20each%20iteration.)

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # IMPORTANT: pick a good Learning Rate for your dataset 
cfg.SOLVER.MAX_ITER = 30000    # 
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only 5 classes ["car", "chair", "cup", "person", and "traffic light"] 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)



## Train the model


In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## Inference & evaluation using the trained model
If you wish to load a trained model to run inference

In [ ]:
evaluator = COCOEvaluator("my_dataset_val", output_dir="../output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

The code below defines a function "format_detectron2_predictions" to convert the prediction output of Detectron2 to a format that can be used by Cleanlab for identifying label errors. This function accepts the predicted object instances and the number of classes as inputs. It processes the predicted bounding boxes and prediction-confidence for each instance, and outputs a list of numpy arrays containing the bounding boxes and prediction-confidence for each class.

In [ ]:
def format_detectron2_predictions(ins,num_classes):
    fields = ins.get_fields()
    boxes = fields['pred_boxes'].tensor.numpy()
    res = [[] for i in range(num_classes)]
    for i in range(0,len(fields['pred_classes'])):
        pred_class = fields['pred_classes'][i].item()
        probs = ins.get_fields()['scores'][i].item()
        box_cord = list(boxes[i])
        box_cord.append(probs)
        res[pred_class].append(box_cord)
    res2 = []
    for i in res:
        if len(i)==0:
            q = np.array(i,dtype=np.float32).reshape((0,num_classes))
        else:
            q = np.array(i,dtype=np.float32)
        res2.append(q)
    return res2
        


To perform inference and testing on the tutorial notebook linked here, we utilize a limited portion of the validation set of COCO 2017. To find label errors in this subset, please run our [tutorial](https://docs.cleanlab.ai/stable/index.html) notebook on [Finding Label Errors in Object Detection Datasets](https://github.com/cleanlab/cleanlab/blob/master/docs/source/tutorials/object_detection.ipynb).

In [ ]:
labels = pickle.load(open("labels.pkl",'rb'))
results = []
for i in labels:
    im_name = os.path.join(VAL_PATH, i['seg_map'].replace(".png",'.jpg'))
    im = cv2.imread(im_name)
    outputs = predictor(im)
    results.append(format_detectron2_predictions(outputs["instances"].to("cpu"),cfg.MODEL.ROI_HEADS.NUM_CLASSES))
    
pickle.dump(results,open("predictions.pkl",'wb'))